In [1]:
import mesa
import numpy as np
from enum import Enum
import uuid
import random

def new_uuid():
    # Generate a UUID
    unique_id = uuid.uuid4()

    # Convert the UUID to an integer
    int_uuid = int(unique_id.int)

    return int_uuid

For SEIR model rigth now we assume, that mosquito can be in SUSCEPTIBLE, EXPOSED or INFECTED state. If mosquito is INFECTED it stays like that for the rest of it's life.

Human after beeing INFECTED for `infection_period` has some probability of recovering. Right now we assume that reovered human is immune to malaria and can be exposed any more.

In [4]:
class SEIR(Enum):
    SUSCEPTIBLE = 1
    EXPOSED = 2
    INFECTED = 3
    RECOVERED = 4

class LIFE_STAGE(Enum):
    LARVAE = 1
    ADULT = 2


class HumanAgent(mesa.Agent):
    """
    If a mosquito bites a human, then the MosquitoAgent is responsible for changing human's seir to EXPOSED. After incubation period
    human's state is changed to INFECTED. After infection_period human has recovery_probability of changing to RECOVERED. If human
    is not recovered then it dies
    """

    def __init__(self, unique_id, model, incubation_period: int, infection_period: int, 
                 recovery_probability: float, seir: SEIR = SEIR.SUSCEPTIBLE):
        super().__init__(unique_id, model)
        self.incubation_period = incubation_period
        self.infection_period = infection_period
        self.recovery_probability = recovery_probability
        self.seir = seir
        self.type = "Human"

    def die(self):
        self.model.schedule.remove(self)
        self.model.grid.remove_agent(self)

    def step(self):
        pass
    

class MosquitoAgent(mesa.Agent):
    """
    Mosquitos can be in one of two life stages LARVAE or ADULT. Only adult mosquitos can move and be malaria vectors.

    If self.looking_for_water is equal to False, then mosquito is looking for human. When mosquito is looking for human it moves one cell
    in random direction until it is in one cell with a human. After biting a human mosquito changes self.looking_for_water to True.
    Now it is moving in random direction to find a water source and it doesn't care about humans.
    
    If it finds cell with other agent of type "Water" it creates some number of new mosquitos in this cell and life stage LARVAE. After
    creating new larves, mosquito changes self.looking_for_water again to False and again searches for human. 

    If mosquito is in one cell with an agent of type "House" and the house has net or spray, it has some chance of being repelled 
    (if mosquito is repelled it doesn't die and doesn't bite, just continue to move). If it wasn't repelled then there is a chance of
    killing a mosquito. If mosquito both isn't repelled and doesn't die, the it can still bite a human if the human is also present in 
    this cell
    
    """

    def __init__(self, unique_id, model, life_time: int, incubation_period: int, larvae_period: int, probability_of_exposition: float,
                 life_stage: LIFE_STAGE = LIFE_STAGE.ADULT, seir: SEIR = SEIR.SUSCEPTIBLE):
        super().__init__(unique_id, model)
        self.current_life_step = 0
        self.life_time = life_time
        self.incubation_period = incubation_period
        self.larvae_period = larvae_period
        self.life_stage = life_stage
        self.seir = seir
        self.type = "Mosquito"
        self.probability_of_exposition = probability_of_exposition
        self.looking_for_water = False

    def move(self):
        #TODO: Only Adult mosquitos can move!!! Move mosquito in random direction
        pass

    def get_human(self):
        #TODO: checks if in the current cell there are any agents with attributr type="Human". 
        #If yes return one of the human agent, if not return null
        pass

    def bite(self, human: HumanAgent):
        #TODO: if the human is INFECTED and mosquito is SUSCEPTIBLE, mosquito has self.probability_of_exposition of changing self.seir 
        # to EXPOSED.
        # If mosquito is INFECTED and human is SUSCEPTIBLE, human.seir is EXPOSED.
        pass

    def die(self):
        self.model.schedule.remove(self)
        self.model.grid.remove_agent(self)

    def check_life_stage(self):
        if self.life_stage == LIFE_STAGE.LARVAE and self.current_life_step >= self.larvae_period:
            # change from larvae to adult
            self.life_stage = LIFE_STAGE.ADULT
        elif self.life_stage == LIFE_STAGE.ADULT and self.current_life_step >= self.life_time:
            self.die()

    def check_seir(self):
        #TODO: if mosquito was EXPOSED, keep track of incubation period in order to change state to INFECTED
        pass

    def step(self):
        self.current_life_step += 1
        self.check_life_stage()
        if self.life_stage == LIFE_STAGE.ADULT:
            #TODO: move in random direction and do stuff described in class description
            pass


class HouseAgent(mesa.Agent):
    """Agent representing a house. It doesn't move"""

    def __init__(self, unique_id, model, mosquito_net: bool, mosquito_spray: bool):
        super().__init__(unique_id, model)
        self.mosquito_net = mosquito_net
        self.mosquito_spray = mosquito_spray
        self.type = "House"
        
    def step(self):
        pass


class WaterAgent(mesa.Agent):
    """Agent representing a water source. It doesn't move and doesn't have any extra properties"""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = "Water"

    def step(self):
        pass

In [8]:
class MalariaInfectionModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, width, height, initial_mosquitos, initial_humans, houses, ponds, percentage_of_infected_humans,
                 human_incubation_period, human_infection_period, human_recovery_probability, 
                 mosquito_incubation_period, mosquito_life_time, mosquito_larvae_period, 
                 mosquito_probability_of_exposition):
        
        self.grid = mesa.space.MultiGrid(width, height, False)
        self.schedule = mesa.time.RandomActivation(self)
        
        # Create human agents
        infected_humans = int(percentage_of_infected_humans*initial_humans)
        susceptible_humans = initial_humans-infected_humans
        for _ in range(infected_humans):
            a = HumanAgent(new_uuid(), self, incubation_period=human_incubation_period, infection_period=human_infection_period, 
                 recovery_probability=human_recovery_probability, seir = SEIR.INFECTED)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
        for _ in range(susceptible_humans):
            a = HumanAgent(new_uuid(), self, incubation_period=human_incubation_period, infection_period=human_infection_period, 
                 recovery_probability=human_recovery_probability, seir = SEIR.SUSCEPTIBLE)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        
        # Create mosquito agents
        for _ in range(initial_mosquitos):
            a = MosquitoAgent(new_uuid(), self, life_time=mosquito_life_time, incubation_period=mosquito_incubation_period, 
                              larvae_period=mosquito_larvae_period, probability_of_exposition=mosquito_probability_of_exposition,
                              life_stage=random.choice(list(LIFE_STAGE)), seir=SEIR.SUSCEPTIBLE)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))


        # Create house agents
        for _ in range(houses):
            a = HouseAgent(new_uuid(), self, random.choice([True, False]), random.choice([True, False]))
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        # Create water agents
        for _ in range(houses):
            a = WaterAgent(new_uuid(), self)
            self.schedule.add(a)

            collision_with_house = True
            
            while collision_with_house:
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                cellmates = self.model.grid.get_cell_list_contents([(x,y)])
                for c in cellmates:
                    if c.type != "House":
                        collision_with_house = False
                        
            self.grid.place_agent(a, (x, y))

    def step(self):
        self.schedule.step()